In [ ]:
# This is a Python block

*Задание**
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. � ассмотреть 2-а варианта сеточек 
2.1 �?нициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 �?нициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
 				
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
import pandas as pd

df_train = pd.read_csv("отзывы за лето.csv", encoding='cp1251')

In [ ]:
df_train.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [2]:
!pip install pymorphy2==0.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=2a3f0ba52fe1f781832a2d82ec2c8a617db796d9fbf0732cc670736950f7295d
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [4]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)


In [6]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [62]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [63]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [64]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [65]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [66]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [67]:
seq = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)

In [69]:
from sklearn.model_selection import train_test_split

In [74]:
num_classes = 5+1
y = keras.utils.to_categorical(df_train["Rating"], num_classes)


In [75]:
X_train, X_test, y_train, y_test = train_test_split(
...     seq, y, test_size=0.33, random_state=42)

In [77]:
y_train[1]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [84]:
y_train[30]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [78]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [79]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [80]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [81]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
verbose=1,
validation_split=0.1,callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 7s 236ms/step - loss: 1.4984 - accuracy: 0.6718 - val_loss: 1.2062 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 5s 214ms/step - loss: 1.1029 - accuracy: 0.7030 - val_loss: 0.9854 - val_accuracy: 0.7040


In [85]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 1s 54ms/step - loss: 0.9548 - accuracy: 0.7130


Test score: 0.9548348188400269
Test accuracy: 0.7129656672477722


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af133a3e-e74e-4642-80b0-22c9c55ad802' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>